In [ ]:
import csv
import os
import importlib

import pandas as pd
import numpy as np
import datetime
import matplotlib.pylab as plt
import seaborn as sns; sns.set()

import utils 
import my_transformers
import const

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin

%matplotlib inline

In [ ]:
importlib.reload(utils)

df = utils.PrepareData().df
df.head()

In [ ]:
importlib.reload(my_transformers)
from my_transformers import DropColumns, YearTransformer, ColumnToDateFormat, Drop33Rooms

transform_pipeline = Pipeline([
        ('yr_built_transformer', YearTransformer(column='yr_built')),
        ('33_bedrooms_row_drop', Drop33Rooms()),
        ('clean_flaw', DropColumns(columns=['id', 'date', 'yr_renovated'])),
        # ('clean_flaw', DropColumns(columns=['bathrooms'])),
        # ('clean_geodata', DropColumns(columns=['zipcode', 'lat', 'long'])),
        ('clean_duplicate', DropColumns(columns=['price'])),
     ])
ndf = transform_pipeline.transform(df)
ndf.head()

In [ ]:
ndf.info()

# Logical Regression

In [ ]:
ndf['price_bin'].hist()

In [ ]:
importlib.reload(utils)
sdf = utils.get_equal_samples(ndf, column='price_bin', mult_coef=1)
sdf['price_bin'].value_counts()

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 5))
sdf['waterfront'].hist(ax=ax)
sdf[sdf['price_bin']==1]['waterfront'].hist(ax=ax)
ax.legend()
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 5))
sdf['view'].hist(ax=ax, label='all sales')
sdf[sdf['price_bin']==1]['view'].hist(ax=ax, label='over 1 ml')
ax.legend()
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 5))
sdf['condition'].hist(ax=ax, label='all sales')
sdf[sdf['price_bin']==1]['condition'].hist(ax=ax, label='over 1 ml')
ax.legend()
plt.show()

In [ ]:
sns.pairplot(ndf[['floors', 'grade', 'bedrooms', 'condition', 'view', 'price_bin']], hue='price_bin', corner=True, size=5);

In [ ]:
sns.pairplot(ndf[['floors', 'grade', 'bedrooms', 'condition', 'view', 'price_bin']], hue='price_bin', corner=True, size=5);sns.pairplot(ndf[['floors', 'grade', 'bedrooms', 'condition', 'view', 'price_bin']], hue='price_bin', corner=True, size=5);

In [ ]:
sdf = ndf.copy()
train_X, test_X, train_y, test_y = train_test_split(
    sdf.drop(columns=['price_bin'], inplace=False), 
    sdf['price_bin'],
    test_size=const.TEST_SIZE,
    random_state=const.RANDOM_STATE
    )

' - '.join([str(getattr(v, 'shape')) for v in (train_X, test_X, train_y, test_y)])

In [ ]:
sgd = SGDClassifier(random_state=const.RANDOM_STATE)
sgd.fit(train_X, train_y)

In [ ]:
cross_val_score(sgd, train_X, train_y, cv=5, scoring='accuracy')
# sgd.predict(test_X)


In [ ]:
train_y_pred = cross_val_predict(sgd, train_X, train_y, cv=5)
precision, recall , threshold = precision_recall_curve(train_y, train_y_pred)

plt.plot(threshold, precision[:-1], 'b', label='precision')
plt.plot(threshold, recall[:-1], 'g', label='recall')
plt.xlabel('threshold')
plt.ylim(0, 1)
plt.show()


In [ ]:
# lr = LogisticRegression(multi_class='multinomial',
#                         solver='lbfgs',
#                         C=8)
lr = LogisticRegression(multi_class='multinomial',
                        solver='lbfgs',
                        C=8)
lr.fit(train_X, train_y)

In [ ]:
train_y_pred = cross_val_predict(lr, train_X, train_y, cv=2)
confusion_matrix(train_y, train_y_pred)

In [ ]:
test_y_pred = lr.predict(test_X)
confusion_matrix(test_y, test_y_pred)

In [ ]:
param_grid = {
    'multi_class': ['auto', 'ovr', 'multinomial' ],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'C': [3, 5, 8, 10, 12]
    }
grid = GridSearchCV(estimator=lr,
                    param_grid=param_grid,
                    scoring='r2',
                    verbose=1,
                    n_jobs=-1)
                    
grid_result = grid.fit(train_X, train_y)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

# KNN

In [ ]:
from sklearn import neighbors, metrics, svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=2, weights='uniform')
knn.fit(train_X, train_y)
predict = knn.predict(test_X)
accuracy = metrics.accuracy_score(test_y, predict)
accuracy

In [ ]:
conf_mat = confusion_matrix(test_y, predict)
conf_mat

# SVM

In [ ]:
_svm = svm.SVC()
_svm.fit(train_X, train_y)
prediction = _svm.predict(test_X)

accuracy = metrics.accuracy_score(test_y, prediction)
print("predictions:", prediction)
print("accuracy: ", accuracy)

In [ ]:
conf_mat = confusion_matrix(test_y, predict)
conf_mat

# PCA

In [ ]:
ndf.columns

In [ ]:
model = PCA(n_components=2)            
model.fit(train_X)                  
X_2D = model.transform(ndf.drop(columns=['price_bin'])) 

In [ ]:
df_pca = ndf.copy()
df_pca['PCA1'] = X_2D[:, 0]
df_pca['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='price_bin', data=df_pca, fit_reg=True)

In [ ]:
model = PCA(n_components=3)            
model.fit(train_X)                  
X_3D = model.transform(ndf.drop(columns=['price_bin'])) 
df_pca['PCA_1'] = X_3D[:, 0]
df_pca['PCA_2'] = X_3D[:, 1]
df_pca['PCA_3'] = X_3D[:, 2]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(6,6))

ax = Axes3D(fig)

ax.scatter(df_pca['PCA_1'], df_pca['PCA_2'], df_pca['PCA_3'], c=df_pca['price_bin'], marker='o', cmap='jet')
ax.set_xlabel('PCA1 Label')
ax.set_ylabel('PCA2 Label')
ax.set_zlabel('PCA3 Label')

plt.show()